In [55]:
import collections
import re
from d2l import torch as d2l

In [56]:
# 读取数据集
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

def read_time_machine():
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

lines = read_time_machine()
print(f'# 文本总行数: {len(lines)}')
print(lines[0])
print(lines[10])

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


In [57]:
# 定义tokenize函数
def tokenize(lines, token='word'):
    if token == 'word':
        return [line.split() for line in lines]         # 按空格分割
    elif token == 'char':
        return [list(line) for line in lines]           # 按字符分割
    else:
        print('错误：未知token类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


In [58]:
"""
构建一个词表，将字符串类型的词元映射到从0开始的数字索引
先将训练集中的文档合并到一起，进行唯一词元的统计，得到的结果为语料
然后根据每一个词元的出现频率，对词元进行排序，最后将词元映射到数字索引（出现很少的词元通常被排除以降低复杂性）
除此之外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元 “<unk>”
"""
class Vocab:
    def __init__(self, tokens=None, reversed_tokens=None, min_freq=0):
        if tokens is None:
            tokens = []
        if reversed_tokens is None:
            reversed_tokens = []
        
        # 计算词频
        """
        count_corpus() 是一个函数，用于统计语料库中每个token出现的次数
        tokens 是输入的分词后的语料库，包含所有文本的词元
        函数返回一个 Counter 对象（来自 Python 的 collections 模块），它类似于字典，键是词元，值是该词元出现的次数
        """
        counter = count_corpus(tokens)
        """
        .items() 方法将 counter 对象转换为列表，其中每个元素都是一个键值对
        sorted() 函数按照词频从高到低对词元进行排序
        lambda x: x[1] 指定了排序的关键字，即词频（lambda是创建一个匿名函数的关键字，x是输入，x[1]表示输入的第二个元素，即词频）
        匿名函数是指没有名字的函数，它允许在不定义函数的情况下创建函数对象
        reverse=True 指定按照词频从高到低排序
        """           
        self.token_freqs = sorted(counter.items(), key=lambda x: x[1], reverse=True)

        # 构建id与token的双向索引，其中“<unk>”标记的索引为0
        self.idx_to_token = ['<unk>'] + [token for token, freq in self.token_freqs if freq >= min_freq]
        self.token_to_idx = {token: idx for idx, token in enumerate(self.idx_to_token)}

        for token, freq in self.token_freqs:
            if freq < min_freq:
                break
            # 如果词元不在词典中，则将其添加到词典中
            if token not in self.token_to_idx:
                self.token_to_idx[token] = len(self.idx_to_token)
                self.idx_to_token.append(token)

    # 获取词典中词元的总数
    def __len__(self):
        return len(self.idx_to_token)
    
    # 递归地将词元转换为索引
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):                               # isinstance() 函数来判断一个对象是否是一个已知的类型，这里判断tokens是否是列表或者元组类型
            return self.token_to_idx.get(tokens, self.token_to_idx['<unk>'])    # 如果tokens不是列表或者元组类型，则返回tokens对应的索引，如果tokens不在词典中，则返回unk对应的索引
        return [self.__getitem__(token) for token in tokens]                    # 如果tokens是列表或者元组类型，则返回tokens中每个元素对应的索引
        
def count_corpus(tokens):
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

In [59]:
# 构建一个词典
vocab = Vocab(tokens)
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


In [60]:
for i in [0, 10]:
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


In [ ]:
def load_corpus_time_machine(max_tokens=-1):
    """返回时光机器数据集的词元索引列表和词表"""
    lines = read_time_machine()
    tokens = tokenize(lines, 'char')
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，所以将所有文本行展平到一个列表中
    corpus = [vocab[token] for line in tokens for token in line]
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    return corpus, vocab

# 获取时光机器数据集的词元索引列表和词表，并返回词元索引列表和词表的长度
corpus, vocab = load_corpus_time_machine()
len(corpus), len(vocab)

(170580, 28)